In [ ]:
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math

In [ ]:
filename1 = 'hw4_train.csv'
df_train = pd.read_csv(filename1)
filename2 = 'hw4_test.csv'
df_test = pd.read_csv(filename2)

In [ ]:
df_test.head(10)


In [ ]:
bins = np.linspace(0, 10,100)
for col_name in list(df_train.columns.values):
    df_temp = df_train.loc[df_train['2.2']==2]
    sns.distplot(df_temp[col_name],  kde=False,label='benign')
    df_temp = df_train.loc[df_train['2.2']==4]
    sns.distplot(df_temp[col_name],  kde=False,label='malignant')
    # Plot formatting
    plt.legend(prop={'size': 12})
    plt.title('Feature name = ' + str(col_name))
    plt.ylabel('Count')
    plt.show()

In [ ]:
print('The number of benign samples are ',end="")
print(len(df_train.loc[df_train['2.2']==2]))
print('The number of malignant samples are ',end="")
print(len(df_train.loc[df_train['2.2']==4]))

In [ ]:
def calculate_conditional_entrophy(df_train):
    
    for col_name in list(df_train.columns.values):
        unique_values = df_train[col_name].unique()
        outcome = 0
        for value in unique_values: 
            benign = len(df_train.loc[(df_train[col_name]==value) & (df_train['2.2']==2)])
            malignant = len(df_train.loc[(df_train[col_name]==value) & (df_train['2.2']==4)])
            if benign == 0:
                log_value_benign =0
                log_value_malicious = math.log2(malignant/(benign+malignant))
            elif malignant == 0:
                log_value_malicious =0
                log_value_benign = math.log2(benign/(benign+malignant))
            else:
                log_value_benign = math.log2(benign/(benign+malignant))
                log_value_malicious = math.log2(malignant/(benign+malignant))
    
            temp = ((benign/(benign+malignant)) * log_value_benign) + ((malignant/(benign+malignant)) * log_value_malicious)
            outcome = outcome + ((benign+malignant)/len(df_train))*abs(temp)
        print('The conditional entrophy of feature {} is {}'.format(col_name,outcome))
calculate_conditional_entrophy(df_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

features = list(df_train.columns.values)
X = df_train[features[:-1]]
Y = df_train['2.2']
X_test = df_test[features[:-1]]
Y_test = df_test['2.1']



def cross_validation(X,Y,leaves,splits,cv=5,scoring='accuracy'):
    for leaf in leaves:
        for split in splits:
            tree_model = DecisionTreeClassifier(max_depth=4,min_samples_leaf=leaf,min_samples_split=split)
            score = cross_val_score(tree_model, X, Y, cv=cv, scoring=scoring)
            print('For tree depth 4 ,min_samples_leaf {} and min_samples_split {}, the score is {} '.format(leaf,split,np.sum(score)/5))




In [ ]:

leaves = range(3,5)
splits = range(2,10)
cross_validation(X,Y,leaves,splits)

In [ ]:
from sklearn.model_selection import GridSearchCV



parameter_grid = {'criterion': ['gini', 'entropy'],\
                  'splitter': ['best', 'random'],\
                  'min_samples_split':range(1,10),\
                  'min_samples_leaf':range(1,5),\
                  'max_depth': [1, 2, 3, 4, 5, 6, 7, 8]}
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X,Y)






In [ ]:
grid = GridSearchCV(decision_tree,param_grid=parameter_grid,cv=5,n_jobs=-1)
grid.fit(X,Y)


In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
model = DecisionTreeClassifier(max_depth=4,min_samples_leaf=3,min_samples_split=4).fit(X,Y)
# score = cross_val_score(tree_model, X, Y, cv=cv, scoring='accuaracy')
# model = DecisionTreeClassifier(max_depth=depth).fit(X_train, y_train)

In [ ]:
accuracy_test = model.score(X_test, Y_test)
print(accuracy_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier



parameter_grid = {'n_estimators':range(10,110,10),
                  'max_depth': [1, 2, 3, 4, 5, 6, 7, 8]}
model = RandomForestClassifier()
model.fit(X,Y)


In [ ]:
grid = GridSearchCV(model,param_grid=parameter_grid,cv=5,n_jobs=-1)
grid.fit(X,Y)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)

In [ ]:
def cross_validation_rf(X,Y, tree_depths,n_estimators,cv=5,scoring='accuracy'):
    for depth in tree_depths:
        for n in n_estimators:
            tree_model = RandomForestClassifier(max_depth=depth,n_estimators=n)
            score = cross_val_score(tree_model, X, Y, cv=cv, scoring=scoring)
            print('For tree depth {} and number of trees {}, the score is {:.2f} '.format(depth,n,(np.sum(score)*100)/5))


tree_depths = range(4,10)
n_estimators = range(10,100,10)
cross_validation_rf(X,Y,tree_depths,n_estimators) 


In [ ]:
model = RandomForestClassifier(max_depth=6,n_estimators=70).fit(X,Y)
accuracy_test = model.score(X_test, Y_test)
print(accuracy_test)